In [1]:
import pandas as pd
from collections import Counter

def split_and_clean_course_code(l):
    if type(l) != str:
        return []
    tmp = l.split(',')
    return [c.replace('-','').strip()[:6] for c in tmp]

In [2]:
Projections = {
                'Old': ['StudentsAudit_old.xlsx','ID','Audit_Courses_to_add'],
                'New': ['StudentsAudit.xlsx','ID','Audit_Courses_to_add']
}

In [7]:
old = pd.read_excel(Projections['Old'][0])[Projections['Old'][1:]]
new = pd.read_excel(Projections['New'][0])[Projections['New'][1:]]
old.columns = ['ID','old']
new.columns = ['ID','new']
old['oID'] = old.ID
new['nID'] = new.ID
df = pd.merge(old, new, on='ID', how='outer')

count = len(df)
count1 = len(df[df.oID.isnull() & df.nID.notnull()])
count2 = len(df[df.oID.notnull() & df.nID.isnull()])
if count1:
    print(f'found {count1} in new but missing from old ({count1/count*100:.2f}%):')
    print(df[df.old.isnull() & df.new.notnull()])
if count2:
    print(f'\nfound {count2} in old but missing from new ({count2/count*100:.2f}%):')
    print(df[df.old.notnull() & df.new.isnull()])
df.drop(['oID','nID'],inplace=True,axis=1)

df.old = df.old.apply(split_and_clean_course_code)
df.new = df.new.apply(split_and_clean_course_code)
df['Delta_old'] = pd.Series(dtype='object')
df['Delta_new'] = pd.Series(dtype='object')
df['Match'] = pd.Series(dtype='float')

for i, r in df.iterrows():
    o = Counter(r.old)
    n = Counter(r.new)
    Delta_o_n = list((o-n).elements())
    Delta_n_o = list((n-o).elements())
    df.at[i,'Delta_old'] = Delta_o_n
    df.at[i,'Delta_new'] = Delta_n_o
    if len(r.old)+len(r.new) == 0:
        df.at[i,'Match'] = 1
    else:
        df.at[i,'Match'] = 1 - (len(Delta_o_n)+len(Delta_n_o)) / (len(r.old)+len(r.new))

df.to_excel('Compare_Projections.xlsx')
# display(df)

print(f'\ncompared {len(df)} students, with similarity ratio: {df.Match.sum()/len(df)*100:.2f}%')
print(f'\nHere are the {len(df[ (df.Delta_old.map(len)>0) | (df.Delta_new.map(len)>0) ])} differences:')
display(df[ (df.Delta_old.map(len)>0) | (df.Delta_new.map(len)>0) ])


compared 1619 students, with similarity ratio: 98.25%

Here are the 103 differences:


,ID,old,new,Delta_old,Delta_new,Match
3,201514581,"[SWE320, CIT466, CIT460, SWE321, SEC330, SEC33...","[SWE320, SWE321, SEC330, SEC331, SEC435, CIT466]","[CIT460, CIT461]",[SEC435],0.769231
11,201619114,"[SEC335, SEC336, NET351, ARA130, ISL135, NET352]","[SEC335, SEC336, NET351, NET352, ARA130]",[ISL135],[],0.909091
53,201806608,"[MTH215, NET256, CIT460, CIT305]","[MTH215, NET256, CIT305, CIT372]",[CIT460],[CIT372],0.750000
79,201814652,"[CIT460, CIT466, CIT461, SEC432, SEC433, INS377]","[CIT460, CIT461, SEC432, SEC433, INS377, ZU_Ele]",[CIT466],[ZU_Ele],0.833333
244,202000970,"[CIT466, CIT460, GroupG, ARA345, INS377, CIT461]","[CIT466, GroupG, ARA345, INS377, INS467, INS469]","[CIT460, CIT461]","[INS467, INS469]",0.666667
...,...,...,...,...,...,...
1321,202322893,"[ENG140, ACC202, CIT365, ARA130, GroupA]","[ENG140, CIT365, ARA130, GroupA, GroupD]",[ACC202],[GroupD],0.800000
1608,M80008838,"[ENG140, CIT460, ARA130, GroupA, GroupD, CIT461]","[ENG140, ARA130, GroupA, GroupD, ISL135]","[CIT460, CIT461]",[ISL135],0.727273
1610,M80008921,"[ENG140, CIT315, SWE245, ARA130, GroupA]","[ENG140, CIT315, NET351, NET352, ARA130, GroupA]",[SWE245],"[NET351, NET352]",0.727273
1614,M80009120,"[ENG140, SWE225, NET256, ARA130, GroupA, NET257]","[ENG140, SWE225, CIT365, ARA130, GroupA]","[NET256, NET257]",[CIT365],0.727273
